## CSE392: Robust Recovery from Tensor (Color Image or Video) Data Collections

In [ ]:
import numpy as np
import cupy as cp
from numpy import linalg as LA
from numpy.linalg import multi_dot
from PIL import Image
import matplotlib.pyplot as plt
from colorsys import hls_to_rgb
import cv2
import time
import pickle
import os
import math

## 0. Datasets

1. [The Berkeley Segmentation Dataset](https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/bsds/)
2. [Youtube](https://www.youtube.com/)

## 2. Related Work

### 2.1 Robust PCA

- Video Surveillance.
- Face Recognition.
- Latent Semantic Indexing.
- Ranking and Collaborative Filtering.

**Key idea**

Denote a matrix $M$ as $M = L_{0} + S_{0}$, where $L_{0}$ is the low-rank representation of $M$ and $S_{0}$ captures the arbitrary corruptions.

Under two weak assumptions, i.e, the rank of the low-rank component is not too large and the sparse component is reasonably sparse, the problem defined above can be converted to the optimization problem,

<center>$minimization\  \left \| L \right \|_{*} + \lambda\left \| S \right \|_{1},\ s.t.\ L+S = M$</center>

The solutions for this optimization problem serve as the best approximation of $L_{0}$ and $S_{0}$.

**Advantages**

- works perfectly to recover the original noise-free matrix with a prior-known $\lambda = \frac{1}{max(n_{1}, n_{2})}$.
- recovers matrix with $rank(L_{0}) = O(\frac{n}{(logn)^{2}})$ and $cardinality(S_{0}) = O(n^{2})$, compared with paper of Chandrasekaran et. al.

**Application (video)**

one dimensional TRPCA

### 2.2 TRPCA

Prox_tnn: The proximal operator of the tensor nuclear norm of a 3 way tensor.

The proximal value for nuclear norm can be denoted as ${\operatorname*{Prox}}_{\lambda \left\| \cdot \right\|_{*}} \left( A \right) = \arg \min_{X} \frac{1}{2} \left\| X - A \right\|_{F}^{2} + \lambda \left\| X \right\|_{*}$

1. Apply SVD on $A$: $A \rightarrow U \operatorname*{diag} \left( \boldsymbol{\sigma} \left( A \right) \right) {V}^{T}$
2. Extract the vector of Singular Values $\boldsymbol{\sigma} \left( A \right)$.
3. Calculate the Proximal Operator of the extracted vector using Vector Norm $p$: $\hat{\boldsymbol{\sigma}} \left( A \right) = {\operatorname*{Prox}}_{\lambda \left\| \cdot \right\|_{p}} \left( \boldsymbol{\sigma} \left( A \right) \right) = \arg \min_{x} \frac{1}{2} \left\| x - \boldsymbol{\sigma} \left( A \right) \right\|_{2}^{2} + \lambda \left\| x \right\|_{p}$
4. Return the Proximal of the Matrix Norm: $\hat{A} = {\operatorname*{Prox}}_{\lambda \left\| \cdot \right\|_{p}} \left( A \right) = U \operatorname*{diag} \left( \hat{\boldsymbol{\sigma}} \left( A \right) \right) {V}^{T}$

**Note:** the paper proposed that tensor-SVD can be efficiently computed based on the matrix SVD in the Fourier domain, since that the block circulant matrix can be mapped to a block diagonal matrix in the Fourier domain.

In [ ]:
def prox_tnn(Y, rho):
    n1, n2, n3 = Y.shape
    X = np.zeros((n1, n2, n3), dtype=complex)
    Y = np.fft.fft(Y)
    tnn = 0
    trank = 0
    U, S, V = np.linalg.svd(Y[:, :, 0], full_matrices=False)
    r = np.sum(S > rho)
    if r >= 1:
        S = S[:r] - rho
        X[:, :, 0] = multi_dot([U[:, :r], np.diag(S), V[:r, :]])
        tnn += np.sum(S)
        trank = max(trank, r)
    halfn3 = round(n3/2)
    for i in range(1, halfn3):
        U, S, V = np.linalg.svd(Y[:, :, i], full_matrices=False)
        r = np.sum(S > rho)
        if r >= 1:
            S = S[:r] - rho
            X[:, :, i] = multi_dot([U[:, :r], np.diag(S), V[:r, :]])
            tnn += np.sum(S)*2
            trank = max(trank, r)
        X[:, :, n3 - i] = np.conjugate(X[:, :, i])
    if n3 % 2 == 0:
        i = halfn3
        U, S, V = np.linalg.svd(Y[:, :, i], full_matrices=False)
        r = np.sum(S > rho)
        if r >= 1:
            S = S[:r] - rho
            X[:, :, i] = multi_dot([U[:, :r], np.diag(S), V[:r, :]])
            tnn += np.sum(S)
            trank = max(trank, r)
    tnn /= 3
    X = np.fft.ifft(X)
    return X, tnn, trank

Prox_l1: The proximal operator of the l1 norm.

Input:
- $x$
- $\lambda$

Output:
- $u$

The proximal value for l1 norm of $u$ can be denoted as $\operatorname{Prox}_{\lambda {\left\| \cdot \right\|}_{1}} \left( x \right) = \arg \min_{u} \left\{ \frac{1}{2} {\left\| u - x \right\|}_{F}^{2} + \lambda {\left\| u \right\|}_{1} \right\}$

The derivative can be calculated and the result will vanishes for $u_{i} = \left\{\begin{matrix}
           x_{i} - \lambda,\ for\ x_{i} > \lambda\\ 
           x_{i} + \lambda,\ for\ x_{i} < -\lambda\\
           0, \ for\ -\lambda \leq x_{i} \leq \lambda
           \end{matrix}\right.$

In [ ]:
def prox_l1(b, plambda):
    return np.maximum(0, b - plambda) + np.minimum(0, b + plambda)

Algorithm TRPCA solved by ADMM:

1. Update $\mathcal{L} _{k+1}$ by the proximal operator of the tensor nuclear norm.
2. Update $\varepsilon _{k+1}$ by the proximal operator of the l1 norm.
3. Update $\mathcal{Y} _{k+1}$ by $\mathcal{L} _{k+1}$, $\varepsilon _{k+1}$ and $\mathcal{X}$.
4. Calculate convergence, if not converge, go to 1.
5. Final $\mathcal{L}$ is the recovery.

In [ ]:
def trpca(X, plambda, tol=1e-8, max_iter=500, rho = 1.1, mu = 1e-4, max_mu = 1e10):
    L = np.zeros(X.shape)
    S = L
    Y = L
    for i in range(max_iter):
        Lk = L
        Sk = S
        L, tnnL, _ = prox_tnn(-S + X - Y/mu, 1/mu)
        S = prox_l1(-L + X - Y/mu, plambda/mu)
        dY = L + S - X
        chgL = np.max(abs(Lk - L))
        chgS = np.max(abs(Sk - S))
        chg = max(chgL, chgS, np.max(abs(dY)))
        if chg < tol:
            break
        Y = Y + mu*dY
        mu = min(rho*mu, max_mu)
        
        obj = tnnL + plambda * LA.norm(S.flatten(), ord=1)
        err = LA.norm(dY.flatten(), ord=2)
        print("Iter: %d/%d     Err: %.4f"%(i+1, max_iter, err))
    return L, S, obj, err, i

In [ ]:
im = Image.open("./Dataset/BSDS300-images/train/12074.jpg")
X = np.array(im).astype("float64")
X = X/255
maxP = np.max(abs(X))
n1, n2, n3 = X.shape
Xn = np.copy(X)
rhos = 0.3
ind = np.random.rand(n1, n2, n3) < rhos
ind[0:100, :, :] = False
ind[200:, :, :] = False    # 100:150, 200:250
ind[:, 0:200, :] = False
ind[:, 300:, :] = False 
Xn[ind] = np.random.rand(np.sum(ind))
mu = 1e-4
max_mu = 1e10
tol = 1e-5
rho = 1.1
max_iter = 500

n1, n2, n3 = Xn.shape
plambda = 1/np.sqrt(max(n1,n2)*n3)

L, S, E, err, i = trpca(Xn, plambda, tol, max_iter, rho, mu, max_mu)

In [ ]:
plt.imshow(X / np.max(X))
plt.show()

In [ ]:
plt.imshow(Xn / np.max(Xn))
plt.show()

In [ ]:
Xhat = L
Xhat = np.maximum(Xhat, 0)
Xhat = np.minimum(Xhat, maxP)
plt.imshow(Xhat.real / np.max(Xhat.real))
plt.axis('off')
plt.savefig('rec6_rpca.png', format='png', dpi=1000)
plt.show()

In [ ]:
print(psnr(Xhat.real, X))

### 2.3 OTPCA

- handle outliers that and delibarate corruptions that are common in practical tensor data
- develop a fast randomized algorithm which requires small sample size but gives great acceleration with no performance drop.

OR-TPCA itself is similar to TRPCA, the only different is that for $\varepsilon$, it solve $\iota_{2,1}$ norm instead of $\iota_{1}$ norm.

<center>$\operatorname{Prox}_{\lambda {\left\| \cdot \right\|}_{1}} \left( x \right) = \arg \min_{u} \left\{ \frac{1}{2} {\left\| u - x \right\|}_{F}^{2} + \lambda {\left\| u \right\|}_{1} \right\}$</center>

**Procedures:**

1. Ramdomly sample a sub-tensor $X \in \mathbb{R}^{n_{1} \times k \times n_{3}}$ from $X$, where $k$ is much smaller than $n_{2}$. Hence, $X = [X_{l}, X_{r}]$, $L_{0} = [L_{l}, L_{r}]$,  $\varepsilon_{0} = [\varepsilon_{l}, \varepsilon_{r}]$. OR-TPCA is first performed on the subset $X_{l}$.
2. Apply $L_{l}$, $E_{l}$ to approximate the value of $L_{r}$ and $E_{r}$.

### 2.4 Corruption-Aware TPCA

focus on the recovery of partial data.

In [ ]:
def rold(X, m, d=5, a=2, b=5, T=0.8):
    n1, n2, n3 = X.shape
    C = np.zeros((n1, n2, n3), dtype=bool)
    V = np.zeros((n1, n2, n3))
    N = d / 2
    for i in range(n1):
        print("%d/%d"%(i+1, n1), end='\r')
        for j in range(n2):
            for t in range(n3):
                lr = int(max(0, i - N))
                rr = int(min(i + N, n1))
                lc = int(max(0, j - N))
                rc = int(min(j + N, n2))
                dst_lst = []
                for r in range(lr, rr):
                    for c in range(lc, rc):
                        if r == i and c == j:
                            continue
                        dst_lst.append(max(math.log(abs(X[r, c, t] - X[i, j, t]) + 1e-8, a), -b) / b + 1)
                rold = np.sum(np.sort(dst_lst)[0:m])
                if rold > T:
                    C[i, j, t] = False      # noise
                    V[i, j, t] = rold
                else:
                    C[i, j, t] = True       # noise-free
    return C, V

In [ ]:
def cor_aw_trpca(X, plambda, C, tol=1e-8, max_iter=500, rho = 1.1, mu = 1e-4, max_mu = 1e10):
    L = np.zeros(X.shape)
    S = L
    Y = L
    for i in range(max_iter):
        Lk = L
        Sk = S
        L, tnnL, _ = prox_tnn(-S + X - Y/mu, 1/mu)
        S = prox_l1(-L + X - Y/mu, plambda/mu)
        S[C] = 0
        dY = L + S - X
        chgL = np.max(abs(Lk - L))
        chgS = np.max(abs(Sk - S))
        chg = max(chgL, chgS, np.max(abs(dY)))
        if chg < tol:
            break
        Y = Y + mu*dY
        mu = min(rho*mu, max_mu)
        
        obj = tnnL + plambda * LA.norm(S.flatten(), ord=1)
        err = LA.norm(dY.flatten(), ord=2)
        print("Iter: %d/%d     Err: %.4f"%(i+1, max_iter, err))
    return L, S, obj, err, i

In [ ]:
im = Image.open("./Dataset/BSDS300-images/train/67079.jpg")
X = np.array(im).astype("float64")
X = X/255
maxP = np.max(abs(X))
n1, n2, n3 = X.shape
Xn = np.copy(X)
rhos = 0.3
ind = np.random.rand(n1, n2, n3) < rhos
ind[0:100, :, :] = False
ind[200:, :, :] = False    # 100:150, 200:250
ind[:, 0:200, :] = False
ind[:, 300:, :] = False 
Xn[ind] = np.random.rand(np.sum(ind))
C, V = rold(Xn, 8, d=5, a=2, b=3, T=0.80)
mu = 1e-4
max_mu = 1e10
tol = 1e-5
rho = 1.1
max_iter = 500
n1, n2, n3 = Xn.shape
plambda = 1/np.sqrt(max(n1,n2)*n3)
L, S, E, err, i = cor_aw_trpca(Xn, plambda, C, tol, max_iter, rho, mu, max_mu)

In [ ]:
plt.imshow(X / np.max(X))
plt.axis('off')
plt.savefig('ori6.png', format='png', dpi=1000)
plt.show()

In [ ]:
plt.imshow(Xn / np.max(Xn))
plt.axis('off')
plt.savefig('cor6.png', format='png', dpi=1000)
plt.show()

In [ ]:
Xhat = L
Xhat = np.maximum(Xhat, 0)
Xhat = np.minimum(Xhat, maxP)
plt.imshow(Xhat.real / np.max(Xhat.real))
plt.axis('off')
plt.savefig('rec6_catrpca.png', format='png', dpi=1000)
plt.show()

In [ ]:
print(psnr(Xhat.real, X))

### 2.5 Evaluation by PSNR

The higher the PSNR, the better the performance.

In [ ]:
def psnr(img1, img2):
    mse = np.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

## 3. Robust Recovery from Videos

Extract frames from videos.

In [ ]:
def frame_capture(path):
    vidObj = cv2.VideoCapture(path)
    count = 0
    success = 1
    while True:
        success, image = vidObj.read()
        if not success:
            break
#         image = image.astype("float64") / 255
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        cv2.imwrite("./Dataset/Videos/italy_short/frame%d.jpg" %(count), image)
        if count == 0:
            X = np.array(image).astype("float64")[:, :, np.newaxis]
        else:
            X = np.concatenate((X, np.array(image).astype("float64")[:, :, np.newaxis]), axis=2)
        count += 1
    return X

In [ ]:
def frame_capture_single(path):
    vidObj = cv2.VideoCapture(path)
    count = 0
    success = 1
    while True:
        success, image = vidObj.read()
        if not success:
            break
#         image = image.astype("float64") / 255
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        cv2.imwrite("./Dataset/Videos/italy_short/frame%d.jpg" %(count), image)
        
        if count == 0:
            X = np.array(image).astype("float64").reshape(image.shape[0]*image.shape[1], 1)
        else:
            X = np.hstack((X, np.array(image).astype("float64").reshape(image.shape[0]*image.shape[1], 1)))
        count += 1
    return X

Convert frames back to videos

In [ ]:
def make_video(output_cor, video_name="./video.mp4", fps=30, size=None):
    height, width = output_cor[:, :, 0].shape
    video = cv2.VideoWriter(video_name, 0, fps, (width, height))
    for idx in range(output_cor.shape[2]):
        layer = np.uint8(output_cor[:, :, idx] * 255)
        X = layer[:, :, np.newaxis]
        X = np.concatenate((X, layer[:, :, np.newaxis]), axis=2)
        X = np.concatenate((X, layer[:, :, np.newaxis]), axis=2)
        video.write(X)
    cv2.destroyAllWindows()
    video.release()

TRPCA

In [ ]:
X = frame_capture("./Dataset/Videos/3_short.mp4")

In [ ]:
X = X/255
maxP = np.max(abs(X))
n1, n2, n3 = X.shape
Xn = np.copy(X)
rhos = 0.3
ind = np.random.rand(n1, n2, n3) < rhos
Xn[ind] = np.random.rand(np.sum(ind))
mu = 1e-4
max_mu = 1e10
tol = 1e-5
rho = 1.1
max_iter = 500
n1, n2, n3 = Xn.shape
plambda = 1/np.sqrt(max(n1,n2)*n3)
L, S, E, err, i = trpca(Xn, plambda, tol, max_iter, rho, mu, max_mu)

RPCA

In [ ]:
# X = frame_capture("./Dataset/Videos/4.mp4")
X = frame_capture("./Dataset/Videos/4.mp4")

In [ ]:
X = X/255
maxP = np.max(abs(X))
n1, n2, n3 = X.shape
Xn = np.copy(X)

In [ ]:
rhos = 0.3
ind = np.random.rand(n1, n2, n3) < rhos
Xn[ind] = np.random.rand(np.sum(ind))
mu = 1e-4
max_mu = 1e10
tol = 1e-5
rho = 1.1
max_iter = 500
n1, n2, n3 = Xn.shape
plambda = 1/np.sqrt(max(n1,n2)*n3)

L = np.zeros(X.shape, dtype=complex)
for i in range(X.shape[2]):
    tmp, S, E, err, j = trpca(Xn[:, :, i][:, :, np.newaxis], plambda, tol, max_iter, rho, mu, max_mu)
    L[:, :, i] = tmp.squeeze()

Corruption-Aware TPCA

In [ ]:
X = frame_capture("./Dataset/Videos/4.mp4")

In [ ]:
X = X/255
maxP = np.max(abs(X))
n1, n2, n3 = X.shape
Xn = np.copy(X)
ind = np.zeros((n1, n2, n3), dtype=bool)
ind[100:125, 200:225, :] = True
Xn[ind] = np.random.rand(np.sum(ind))
C, V = rold(Xn, 8, d=5, a=2, b=3, T=0.9)

In [ ]:
mu = 1e-4
max_mu = 1e10
tol = 1e-5
rho = 1.1
max_iter = 500
n1, n2, n3 = Xn.shape
plambda = 1/np.sqrt(max(n1,n2)*n3)
L, S, E, err, i = cor_aw_trpca(Xn, plambda, C, tol, max_iter, rho, mu, max_mu)

## References

1. [Robust Principal Component Analysis?](https://statweb.stanford.edu/~candes/papers/RobustPCA.pdf)
2. [Tensor Robust Principal Component Analysis: Exact Recovery of Corrupted Low-Rank Tensors via Convex Optimization](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Lu_Tensor_Robust_Principal_CVPR_2016_paper.pdf)